In [1]:
from typing import List, Optional

import fire

from llama import Dialog, Llama
import torch.distributed as dist
import torch
import os
from data_casual import output_list_test, input_list_test
from llama.model import ModelArgs, Transformer
from llama.tokenizer import ChatFormat, Dialog, Message, Tokenizer
import json
import os
import sys
import time
from pathlib import Path
from typing import List, Optional, Tuple, TypedDict

from eval import calc_loss_steering_vector, extracting_steering_vector

import torch
import torch.nn.functional as F
from fairscale.nn.model_parallel.initialize import (
    get_model_parallel_rank,
    initialize_model_parallel,
    model_parallel_is_initialized,
)

In [2]:
steering_vec = torch.load("./steering_without_instruct/n_vector26.pt")


/tmp/ipykernel_27246/270507088.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  steering_vec = torch.load("./steering_without_instruct/n_vector26.pt")


In [3]:
generator = Llama.build("./", "./tokenizer.model",max_seq_len= 1024,max_batch_size= 4, activation=True, activation_layer=26,steering_vector=steering_vec)

/home/ribbee02/VSC/llama3/llama/generation.py:98: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location="cpu")


> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1


/home/ribbee02/VSC/Coding_venv/lib/python3.11/site-packages/torch/__init__.py:1144: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:432.)
  _C._set_default_tensor_type(t)


Loaded in 16.60 seconds


In [4]:
ckpt_dir = "./"
tokenizer_path = "./tokenizer.model"
temperature = 0.6
top_p = 0.9
max_seq_len = 1024
max_batch_size = 4
max_gen_len= 1

In [5]:
loss = calc_loss_steering_vector(generator, steering_vec, (input_list_test, output_list_test), layer=26, iter=len(input_list_test), multiplier=14)
print(f"Layer {26} loss with multiplier {15} is {loss}")

DDDDDDDDDDBBBCCCCBDDDCDDCDDDCCCCBDDDDDCCADDDDDDCDCDDDDDDDDDDDDDDDDDCCDCCCDDCDDDDDDDDDDDDCCDDDDDDCDCDDCDCDCCCDC
Layer 26 loss with multiplier 15 is 0.4199655765920782


In [13]:
generator.change_activation_vector(14.5*steering_vec)

In [14]:
generator.change_activation_layer(26)

In [15]:
value, activations = generator.chat_completion([input_list_test[1142]],
                                   max_gen_len=max_gen_len,
                                   top_p=top_p,
                                   temperature=temperature)
print(value, activations)

n = len(input_list_test)
values = []
for i in range(n):
    value, activations = generator.chat_completion([input_list_test[i]],
                                      max_gen_len=max_gen_len,
                                       top_p=top_p,
                                       temperature=temperature)
    value2 = 1 if value[0]["generation"]["content"]=="T" else 0
    values.append(value2)

[{'generation': {'role': 'assistant', 'content': 'F'}}] [None, None]


In [26]:
output_list_test.count("0")

982